In [2]:
import pandas as pd
import numpy as np
import scipy.sparse as sp
import pickle
import os

## Basic Association 

In [3]:
orders = pd.read_csv('src/data/order_products__prior.csv')

In [4]:
def construct_dic_matrix(df):
    order_groups = orders.groupby('order_id')
    matrix = {}
    for order_id in order_groups.groups.keys():
        cart = order_groups.get_group(order_id)['product_id'].sort_values().to_list()
        for i, el in enumerate(cart):
            if el not in matrix:
                matrix[el] = {}
            for j in cart[(i+1):]:
                matrix[el][j] = matrix[el].get(j, 0) + 1

    delkeys = []
    for i in matrix.keys():
        for k,v in matrix[i].items():
            if v == 1:
                delkeys.append((i,k))

    for i,j in delkeys:
        del matrix[i][j]

    return matrix

In [5]:
def to_sparseMatrix(matrix):
    mat = sp.dok_matrix((49689 , 49689), dtype=np.int16)

    for ix1, ixx2 in matrix.items():
        for ix2 in ixx2:
            mat[ix1, ix2] = ixx2[ix2]

    mat = mat.transpose().tocsr()
    return mat

In [6]:
%%time
MATRIX = construct_dic_matrix(orders)

CPU times: total: 13min 35s
Wall time: 13min 38s


In [7]:
sparse_matrix = to_sparseMatrix(MATRIX)

In [9]:
!pip install neptune

     ---------------------------------------- 0.0/450.4 kB ? eta -:--:--
     ------------------------------------- 450.4/450.4 kB 27.5 MB/s eta 0:00:00
  Using cached click-8.1.3-py3-none-any.whl (96 kB)
     ---------------------------------------- 0.0/135.9 kB ? eta -:--:--
     ---------------------------------------- 135.9/135.9 kB ? eta 0:00:00
  Using cached oauthlib-3.2.2-py3-none-any.whl (151 kB)
  Using cached swagger_spec_validator-3.0.3-py2.py3-none-any.whl (27 kB)
  Using cached bravado-11.0.3-py2.py3-none-any.whl (38 kB)
  Using cached GitPython-3.1.31-py3-none-any.whl (184 kB)
  Using cached future-0.18.3.tar.gz (840 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached requests_oauthlib-1.3.1-py2.py3-none-any.whl (23 kB)
     ---------------------------------------- 0.0/11.0 MB ? eta -:--:--
     ------------ --------------------------- 3.3/11.0 MB 69.9 MB/s eta 0:00:01
     -----------------------------

In [8]:
with open('output/BasicAsso.pkl', 'wb') as f:
    pickle.dump(sparse_matrix, f)

In [11]:
import neptune

PROJECT="yassine-mhe/CompleteBasket"
NAME="blabla"
KEY="keyblala"
API_TOKEN="api_token"

In [13]:
model = neptune.init_model(
    name=NAME,
    key=KEY, 
    project=PROJECT, 
    api_token=API_TOKEN
)

https://app.neptune.ai/yassine-mhe/CompleteBasket/m/COM-MOD


In [14]:
model["model/specs"].upload('output/BasicAsso.pkl')

## Apriori Algorithm